In [1]:
import pandas as pd
import numpy as np
from tqdm import trange

In [2]:
col_names=['广告id','创建时间','广告账户id','商品id','商品类型','广告行业id','素材尺寸','创建/修改时间','操作类型','修改字段','操作后的字段值']
path='/home/htc/code/Github/TencentAlgo/ad_static_operation.csv'
data=pd.read_csv(path,low_memory=False,encoding='utf-8')

In [23]:
def add_time_stamp(data):
    p1=lambda x:x.replace('0b','')
    p2=lambda x:x[::-1]
    p3=lambda x:x+'0'*(48-len(list(x)))
    time_stamp=['00:00~00:30','00:30~01:00',
                   '01:00~01:30','01:30~02:00',
                   '02:00~02:30','02:30~03:00',
                   '03:00~03:30','03:30~04:00',
                   '04:00~04:30','04:30~05:00',
                   '05:00~05:30','05:30~06:00',
                   '06:00~06:30','06:30~07:00',
                   '07:00~07:30','07:30~08:00',
                   '08:00~08:30','08:30~09:00',
                   '09:00~09:30','09:30~10:00',
                   '10:00~10:30','10:30~11:00',
                   '11:00~11:30','11:30~12:00',
                   '12:00~12:30','12:30~13:00',
                   '13:00~13:30','13:30~14:00',
                   '14:00~14:30','14:30~15:00',
                    '15:00~15:30','15:30~16:00',
                    '16:00~16:30','16:30~17:00',
                    '17:00~17:30','17:30~18:00',
                    '18:00~18:30','18:30~19:00',
                    '19:00~19:30','19:30~20:00',
                    '20:00~20:30','20:30~21:00',
                    '21:00~21:30','21:30~22:00',
                    '22:00~22:30','22:30~23:00',
                    '23:00~23:30','23:30~24:00'
                   ]
    time_stamp=np.asarray(time_stamp*7)
    week=[['Mon:']*48,['Tues:']*48,['Wed:']*48,['Thurs:']*48,['Fri:']*48,['Sat:']*48,['Sun:']*48]
    week=np.reshape(week,(336))
    p2=lambda x,y:x+y
    final_time=list(map(p2,week,time_stamp))
    p=lambda x:x.replace('0b','')[::-1]+'0'*(48-len(list(x.replace('0b','')[::-1])))
    final_index=['广告id','商品id']+final_time
    final_index=np.asarray(final_index)
    out=pd.DataFrame(columns=final_index)
    
    for i in trange(len(data)):
        temp=data.iloc[i]
        if temp['修改字段']==4 and len(list(temp['操作后的字段值'].split(',')))==7:
            num=temp['操作后的字段值']
            num=list(map(p,list(map(bin,list(map(int,list(num.split(','))))))))
            tmp_str=''
            for j in num:
                tmp_str+=j
            tmp_str=list(map(int,list(tmp_str)))
            tmp_str=[temp['广告id'],temp['商品id']]+tmp_str
            tmp_str=np.asarray(tmp_str)
            tmp_str=np.expand_dims(tmp_str,0)
            final_df=pd.DataFrame(tmp_str,columns=final_index,dtype=int)
            out=out.append(final_df,ignore_index=True)
    return out


In [24]:
out=add_time_stamp(data)
print(out.head())

100%|██████████| 614182/614182 [30:29<00:00, 335.77it/s]  

     广告id   商品id Mon:00:00~00:30 Mon:00:30~01:00 Mon:01:00~01:30  \
0  347637  28775               0               0               0   
1  288207  17911               0               0               0   
2  281688   4000               0               0               0   
3  449295  20007               0               0               0   
4  327801  31081               0               0               0   

  Mon:01:30~02:00 Mon:02:00~02:30 Mon:02:30~03:00 Mon:03:00~03:30  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               0               0   
3               0               0               0               0   
4               0               0               0               0   

  Mon:03:30~04:00  ... Sun:19:00~19:30 Sun:19:30~20:00 Sun:20:00~20:30  \
0               0  ...               1               1               1   
1               0  ...      

In [25]:
out.to_csv('timestamp.csv', index=None, encoding='utf-8')